# ODE Refresher

Dan Segal

-----

In [ ]:
using BoundaryValueDiffEq
using Suppressor
using Interact
using Plots

pyplot()

## Problem I

logic: treat as system of 1st order odes

------

$
\ddot x + a \sin x = b \cos \omega t \sin x
$

$
t \in [0,2\pi]
$

$
\rightarrow
$

$
\dot x = v
$

$
\dot v = b \cos \omega t \sin x - a \sin x
$


In [ ]:
t_l = 0
t_r = 2 * pi

@manipulate for x_l = linspace(-1,1,11), x_r = linspace(-1,3,11), a = linspace(-2,3,21), b = linspace(-2,4,21), w = map(ww-> round(ww,2),[ ( 1./(4:-1:1) )..., (1:3)... ])
    cur_ode = function (du,u,p,t)
        du[1] = u[2]
        du[2] = sin(u[1]) * ( b * cos( w * t ) - a )
    end

    boundary_conditions = function (residual, u, p, t)
        residual[1] = u[1][1] - x_l
        residual[2] = u[end][1] - x_r
    end

    cur_problem = TwoPointBVProblem(cur_ode, boundary_conditions, ( rand(2)*2-1 ), (t_l, t_r))
    cur_solution = @suppress solve(cur_problem, MIRK4(), dt=(t_r-t_l)/21)

    plot(xlabel="t",title="Prob I: Driven Pendulum")
    plot!(cur_solution.t, map(first,cur_solution.u), label="x")
    plot!(cur_solution.t, map(last,cur_solution.u), label="v")
end

## Problem II

logic: cast problem as simple matrix solve

------

$
( 1 - t^2 ) \ddot x - 2 t \dot x + n ( n + 1 ) x = \cos t
$

$
t \in [-1,1]
$


$
\rightarrow
$

$
( 1 - t_i^2 ) \cdot \left( \frac{x_{i-1} - 2 x_{i} + x_{i+1}}{\Delta t^2} \right) - t_i \cdot \left( \frac{ x_{i+1} - x_{i-1} }{\Delta t} \right) + n ( n + 1 ) x_i = \cos t_i
$

In [ ]:
t_l = -1
t_r = +1

@manipulate for x_l = linspace(-1,1,11), x_r = linspace(-1,3,11), n = 0:5, N = map(Int,logspace(1,4,4)/2)
    dt = ( t_r - t_l ) / ( N + 1 )

    A = zeros(N,N)
    b = zeros(N)

    T = (i) -> ( t_l + i * dt )

    L = (i) -> (+1/dt^2) * ( 1 - T(i)^2 ) + T(i) / dt
    R = (i) -> (+1/dt^2) * ( 1 - T(i)^2 ) - T(i) / dt

    M = (i) -> (-2/dt^2) * ( 1 - T(i)^2 ) + n * (n+1)

    for i in 1:N
        ( i > 1 ) && ( A[i,i-1] = L(i) )
        A[i,i] = M(i)
        ( i < N ) && ( A[i,i+1] = R(i) )
    end

    b[1] -= L(1) * x_l
    b[end] -= R(N) * x_r

    tt = [ t_l ; ( T.(1:N) )... ; t_r ]
    xx = [ x_l ; ( A \ b )... ; x_r ]

    plot(tt,xx,xlabel="t",ylabel="x",label="", title="Prob II: Linear ODE")
end

## Problem III


logic: handle non-linearity by marching in $t$

------

$
x^p \ddot x - 2 t \dot x + n ( n + 1 ) x = \cos t
$

$
t \in [-1,1]
$


$
\rightarrow
$

$
x_1 = - \frac{ \cos t_0 }{t_0} \cdot \frac{\Delta t}{2}
$

$
x_{i_1} = \frac{ \cos t_i \Delta t^2 - x_i \cdot ( n ( n + 1 ) \Delta t^2 - 2 x_i^p ) - x_{i-1} \cdot ( x_i^p + t_i \Delta t ) }{ ( x_i^p - t_i \Delta t ) }
$

In [ ]:
t_l = -1
t_r = +1

@manipulate for p = map(ww-> round(ww,2),[ ( 1./(4:-1:1) )..., (1:3)... ]), n = 0:5, N = map(Int,logspace(1,4,4)/2)

    dt = ( t_r - t_l ) / ( N + 1 )

    tt = collect(t_l:dt:t_r)

    xx = zeros(N+2)

    xx[1] = 0
    xx[2] = - cos(tt[1])/tt[1] * dt/2

    for i in 3:N+2
        xx[i] = cos(tt[i-1]) * dt^2
        xx[i] -= xx[i-1] * ( n * (n+1) * dt^2 - 2 * xx[i-1]^p )
        xx[i] -= xx[i-2] * ( xx[i-1]^p + tt[i-1] * dt )
        xx[i] /= ( xx[i-1]^p - tt[i-1] * dt )
    end

    min_t, max_t = minimum(tt), maximum(tt)

    bad_indices = append!(find(iszero, xx), find(iszero, tt))
    ttt = deleteat!(deepcopy(tt),bad_indices)
    xxx = deleteat!(map(abs,deepcopy(xx)),bad_indices)

    plot(
        plot(tt,xx, ylims=[-2,2], title="Prob III:", label="x", xlabel="t"),
        plot(ttt,xxx,yscale=:log10, title="Vanishing Term", label="log |x|", xlabel="t")
    )
    
end